In [24]:
import gurobipy as gp
from gurobipy import GRB

In [25]:
# Data from the snippet
probabilities = [0.09, 0.12, 0.10, 0.05, 0.16, 0.14, 0.03, 0.08, 0.05, 0.05, 0.04, 0.03, 0.02, 0.01, 0.02, 0.01]
demands = [90, 95, 100, 105, 110, 115, 120, 125, 130, 135, 140, 145, 150, 155, 160, 165]
supplier_costs = [120, 105, 110]  # Costs for Phil & Sebastian, Rosso, and Monogram respectively
advance_order_cost = 95
min_order_quantities = [0, 70, 40]  # Minimum order quantities for Phil & Sebastian, Rosso, and Monogram

In [26]:
# Model setup
m = gp.Model("Coffee_Supply")

In [27]:
# Decision variables
x = m.addVar(name="x", vtype=GRB.CONTINUOUS)  # Advanced order
y = m.addVars(3, 16, vtype=GRB.CONTINUOUS, name="y")  # Emergency orders

In [28]:
# Objective: Minimize the expected cost
m.setObjective(advance_order_cost * x + gp.quicksum(probabilities[n] * gp.quicksum(supplier_costs[i] * y[i, n] for i in range(3)) for n in range(16)), GRB.MINIMIZE)

In [29]:
# Constraints
# Demand satisfaction in each scenario
for n in range(16):
    m.addConstr(x + gp.quicksum(y[i, n] for i in range(3)) >= demands[n], name=f"demand_satisfaction_{n}")

# Minimum order constraints for Rosso (index 1) and Monogram (index 2)
for i in [1, 2]:  # We do not need a minimum constraint for Phil & Sebastian (index 0)
    for n in range(16):
        is_ordered = m.addVar(vtype=GRB.BINARY, name=f"is_ordered_{i}_{n}")
        m.addConstr(y[i, n] >= min_order_quantities[i] * is_ordered, name=f"min_order_{i}_{n}")
        m.addConstr(y[i, n] <= demands[n] * is_ordered, name=f"max_order_{i}_{n}")

In [30]:
# Solve the model
m.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.0.0 23A344)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 80 rows, 81 columns and 192 nonzeros
Model fingerprint: 0xfe83bfe6
Variable types: 49 continuous, 32 integer (32 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  Objective range  [1e+00, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [9e+01, 2e+02]
Found heuristic solution: objective 13758.000000
Presolve time: 0.00s
Presolved: 80 rows, 81 columns, 192 nonzeros
Variable types: 49 continuous, 32 integer (32 binary)

Root relaxation: objective 1.113550e+04, 29 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 11135.5000    0   13 13758.0000 11135.5000  19.1%     -    0s
H    0     0                  

In [31]:
# Output solution
if m.status == GRB.OPTIMAL:
    print(f"Optimal amount of coffee to order in advance: {x.X} gallons at a cost of ${advance_order_cost * x.X}.")
    for n in range(16):
        for i in range(3):
            if y[i, n].X > 0:
                print(f"Order {y[i, n].X} gallons from supplier {i+1} in scenario {n+1} at a cost of ${supplier_costs[i] * y[i, n].X}.")
else:
    print("No optimal solution found.")

Optimal amount of coffee to order in advance: 95.0 gallons at a cost of $9025.0.
Order 5.0 gallons from supplier 1 in scenario 3 at a cost of $600.0.
Order 10.0 gallons from supplier 1 in scenario 4 at a cost of $1200.0.
Order 15.0 gallons from supplier 1 in scenario 5 at a cost of $1800.0.
Order 20.0 gallons from supplier 1 in scenario 6 at a cost of $2400.0.
Order 25.0 gallons from supplier 1 in scenario 7 at a cost of $3000.0.
Order 30.0 gallons from supplier 1 in scenario 8 at a cost of $3600.0.
Order 35.0 gallons from supplier 1 in scenario 9 at a cost of $4200.0.
Order 40.0 gallons from supplier 3 in scenario 10 at a cost of $4400.0.
Order 45.0 gallons from supplier 3 in scenario 11 at a cost of $4950.0.
Order 50.0 gallons from supplier 3 in scenario 12 at a cost of $5500.0.
Order 55.0 gallons from supplier 3 in scenario 13 at a cost of $6050.0.
Order 60.0 gallons from supplier 3 in scenario 14 at a cost of $6600.0.
Order 65.0 gallons from supplier 3 in scenario 15 at a cost of $

In [32]:
# Assuming `m` is the Gurobi model after optimization has been performed
print(f"Optimal amount of coffee to order in advance: {x.X} gallons.")
print(f"Optimal objective function value (total expected cost): ${m.ObjVal:.2f}.")


Optimal amount of coffee to order in advance: 95.0 gallons.
Optimal objective function value (total expected cost): $11343.50.


In [39]:
# Constants
emergency_costs = [120, 105, 110]  # Phil & Sebastian, Rosso, Monogram

# Cheapest reliable emergency option without minimum exceeding common additional demands
cheapest_reliable_emergency_cost = min(emergency_costs)  # Assuming no constraints for simplicity

# Print the break-even pre-order price
print(f"The break-even pre-order price: ${cheapest_reliable_emergency_cost:.2f} per gallon")

The break-even pre-order price: $105.00 per gallon


In [36]:
# Determine the price per gallon to order the maximum amount in advance
# This is based on the scenario with the highest demand and its emergency order cost

# Find the highest demand scenario
max_demand_scenario = max(demands)
max_demand_cost = max_demand_scenario * supplier_costs[2]  # Assuming supplier 3 has the cheapest emergency cost

# Now we find the price per gallon where this is equivalent to the cost of ordering for the max demand scenario
price_per_gallon_max_advance = max_demand_cost / max_demand_scenario
print(f"Price per gallon to order the maximum amount in advance: ${round(price_per_gallon_max_advance, 2)}")


Price per gallon to order the maximum amount in advance: $110.0


In [47]:
import gurobipy as gp
from gurobipy import GRB

# Given data and setup
probabilities = [0.09, 0.12, 0.10, 0.05, 0.16, 0.14, 0.03, 0.08, 0.05, 0.05, 0.04, 0.03, 0.02, 0.01, 0.02, 0.01]
demands = [90, 95, 100, 105, 110, 115, 120, 125, 130, 135, 140, 145, 150, 155, 160, 165]
supplier_costs = [120, 105, 110]  # Costs for Phil & Sebastian, Rosso, and Monogram respectively
min_orders = [0, 70, 40]  # Minimum orders for Rosso and Monogram

# Assuming that you have already optimized your model and have the model's objective value:
expected_cost_stochastic = 11343.50  # Use model.objVal if available after optimization

# Calculate the optimal cost under perfect information
perfect_info_costs = []
for demand in demands:
    # Calculate minimum cost options for each supplier given the demand
    costs = []
    # Phil & Sebastian (no minimum order)
    costs.append(demand * supplier_costs[0])
    # Rosso (minimum order 70)
    if demand >= min_orders[1]:
        costs.append(demand * supplier_costs[1])
    # Monogram (minimum order 40)
    if demand >= min_orders[2]:
        costs.append(demand * supplier_costs[2])
    
    # Store the minimum cost for this demand
    perfect_info_costs.append(min(costs))

# Weighted average cost under perfect information
perfect_information_cost = sum(p * c for p, c in zip(probabilities, perfect_info_costs))

# Calculate EVPI using the absolute value to ensure non-negative output
evpi = abs(expected_cost_stochastic - perfect_information_cost)
print(f"Expected Value of Perfect Information (EVPI): ${evpi:.2f}")

Expected Value of Perfect Information (EVPI): $694.75


In [48]:
# Calculate the average demand
average_demand = sum(p * d for p, d in zip(probabilities, demands))

# Calculate the EEV assuming the cheapest cost applies uniformly to the average demand
eev_costs = min([average_demand * cost if average_demand >= min_order else float('inf') 
                 for cost, min_order in zip(supplier_costs, min_orders)])
eev_cost = eev_costs  # Cost when assuming average demand is known and constant

# VSS is the difference between the expected cost under the stochastic model and EEV
vss = expected_cost_stochastic - eev_cost

print(f"Expected Value from the Mean Value Problem (EEV): ${eev_cost:.2f}")
print(f"Value of Stochastic Solution (VSS): ${vss:.2f}")

Expected Value from the Mean Value Problem (EEV): $12038.25
Value of Stochastic Solution (VSS): $-694.75


In [49]:
import gurobipy as gp
from gurobipy import GRB

# Data from the snippet
probabilities = [0.09, 0.12, 0.10, 0.05, 0.16, 0.14, 0.03, 0.08, 0.05, 0.05, 0.04, 0.03, 0.02, 0.01, 0.02, 0.01]
demands = [90, 95, 100, 105, 110, 115, 120, 125, 130, 135, 140, 145, 150, 155, 160, 165]
supplier_costs = [120, 105, 110]  # Costs for Phil & Sebastian, Rosso, and Monogram respectively
min_order_quantities = [0, 70, 40]  # Minimum order quantities for Rosso and Monogram

# Perfect foresight models for each scenario
perfect_foresight_costs = []
for n, demand in enumerate(demands):
    m_pf = gp.Model("Coffee_Supply_Perfect_Foresight_" + str(n))
    
    # Decision variables
    y = m_pf.addVars(3, vtype=GRB.INTEGER, name=f"y_{n}")  # Emergency orders from each supplier
    use = m_pf.addVars(2, vtype=GRB.BINARY, name=f"use_{n}")  # Binary variables for Rosso and Monogram

    # Objective function to minimize cost for this scenario
    total_cost_expr = gp.quicksum(supplier_costs[i] * y[i] for i in range(3))
    m_pf.setObjective(total_cost_expr, GRB.MINIMIZE)

    # Constraints
    m_pf.addConstr(gp.quicksum(y[i] for i in range(3)) >= demand, name=f"Demand_{n}")

    # Rosso Coffee minimum order constraints
    m_pf.addConstr(y[1] >= 70 * use[0], name=f"MinOrder_Rosso_{n}")
    m_pf.addGenConstrIndicator(use[0], True, y[1] >= 70, name=f"Activate_Rosso_{n}")
    m_pf.addGenConstrIndicator(use[0], False, y[1] == 0, name=f"Deactivate_Rosso_{n}")

    # Monogram Coffee minimum order constraints
    m_pf.addConstr(y[2] >= 40 * use[1], name=f"MinOrder_Monogram_{n}")
    m_pf.addGenConstrIndicator(use[1], True, y[2] >= 40, name=f"Activate_Monogram_{n}")
    m_pf.addGenConstrIndicator(use[1], False, y[2] == 0, name=f"Deactivate_Monogram_{n}")

    # Solve the model
    m_pf.optimize()

    # Store costs for each scenario, weighted by scenario probability
    if m_pf.status == GRB.OPTIMAL:
        perfect_foresight_costs.append(m_pf.objVal * probabilities[n])

# Calculate WS (Wait-and-See)
WS = sum(perfect_foresight_costs)

# Stochastic Programming Solution (SP) from your original calculation
SP = 11343.5  # Replace with m.objVal if available

# Calculate EVPI
EVPI = SP - WS

print("SP (Stochastic Programming Solution):", SP)
print("WS (Wait-and-See Solution):", WS)
print("EVPI (Expected Value of Perfect Information):", EVPI)

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.0.0 23A344)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 3 rows, 5 columns and 7 nonzeros
Model fingerprint: 0x96ed5b77
Model has 4 general constraints
Variable types: 0 continuous, 5 integer (2 binary)
Coefficient statistics:
  Matrix range     [1e+00, 7e+01]
  Objective range  [1e+02, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [9e+01, 9e+01]
  GenCon rhs range [4e+01, 7e+01]
  GenCon coe range [1e+00, 1e+00]
Presolve added 2 rows and 0 columns
Presolve time: 0.00s
Presolved: 5 rows, 5 columns, 11 nonzeros
Variable types: 0 continuous, 5 integer (2 binary)
Found heuristic solution: objective 10800.000000
Found heuristic solution: objective 9900.0000000
Found heuristic solution: objective 9450.0000000

Root relaxation: cutoff, 0 iterations, 0.00 seconds (0.00 work units)

Explored 1 nodes (0 simplex iterations) in 0.02 seco

In [51]:
import gurobipy as gp
from gurobipy import GRB


# Calculate mean demand
mean_demand = sum(s['p'] * s['d'] for s in probabilities)

# Model setup for the deterministic mean value problem
m_mean = gp.Model("Deterministic_Mean_Value_Problem")

# Decision variables
x_mean = m_mean.addVar(name="x_mean", vtype=GRB.INTEGER)  # Base order

# Objective function to minimize cost
m_mean.setObjective(95 * x_mean, GRB.MINIMIZE)

# Constraint to ensure demand is met using average demand
m_mean.addConstr(x_mean >= mean_demand, "AverageDemand")

# Solve the model
m_mean.optimize()

print("Objective for the Average:", m_mean.objVal)

#Step 2: stochastic solution holding first-stage variables fixed
# Create a new optimization model
m_stochastic = gp.Model("Stochastic_Model")

# Use first-stage decisions from the mean value problem
fs_x = x_mean.X

# Scenario-specific decision variables
y1 = m_stochastic.addVars(len(probabilities), vtype=GRB.INTEGER, name="y1")
y2 = m_stochastic.addVars(len(probabilities), vtype=GRB.INTEGER, name="y2")
y3 = m_stochastic.addVars(len(probabilities), vtype=GRB.INTEGER, name="y3")

# Objective function to minimize total expected cost
objective = gp.quicksum(s['p'] * (95 * fs_x + 120 * y1[i] + 105 * y2[i] + 110 * y3[i]) for i, s in enumerate(scenarios))
m_stochastic.setObjective(objective, GRB.MINIMIZE)

# Constraints for each scenario
for i, s in enumerate(probabilities):
    m_stochastic.addConstr(fs_x + y1[i] + y2[i] + y3[i] >= s['d'], name=f"Demand_{i}")

# Additional constraints for minimum orders if any

# Solve our model
m_stochastic.optimize()

print("Objective for the Stochastic Solution:", m_stochastic.objVal)

# EEV from the mean value problem
EEV = m_mean.objVal

# SP from the stochastic model
SP = m_stochastic.objVal

# Calculate VSS
VSS = abs(EEV - SP)

print("EEV (Expected Ex-Post Value):", EEV)
print("SP (Expected Value of Stochastic Solution):", SP)
print("VSS (Value of Stochastic Solution):", VSS)

TypeError: 'float' object is not subscriptable

In [52]:
# Perfect foresight models
perfect_foresight_costs = []

for s in probabilities:
    m = gp.Model("Coffee_Supply_Perfect_Foresight_" + str(s['n']))

    # Decision variables
    x = m.addVar(name="x", vtype=GRB.INTEGER)  # Base order
    y1 = m.addVar(name="y1", vtype=GRB.INTEGER)  # Emergency order from Phil & Sebastian
    y2 = m.addVar(name="y2", vtype=GRB.INTEGER)  # Emergency order from Rosso
    y3 = m.addVar(name="y3", vtype=GRB.INTEGER)  # Emergency order from Monogram
    use_y2 = m.addVar(name="use_y2", vtype=GRB.BINARY)  # Whether to order from Rosso
    use_y3 = m.addVar(name="use_y3", vtype=GRB.BINARY)  # Whether to order from Monogram

    # Objective function to minimize cost for this scenario
    m.setObjective(95 * x + 120 * y1 + 105 * y2 + 110 * y3, GRB.MINIMIZE)

    # Constraints
    m.addConstr(x + y1 + y2 + y3 >= s['d'], name="Demand")

    # Rosso Coffee minimum order constraints
    m.addConstr(y2 >= 70 * use_y2, name="MinOrder_Rosso")
    m.addGenConstrIndicator(use_y2, True, y2 >= 70, name="Activate_Rosso")
    m.addGenConstrIndicator(use_y2, False, y2 == 0, name="Deactivate_Rosso")

    # Monogram Coffee minimum order constraints
    m.addConstr(y3 >= 40 * use_y3, name="MinOrder_Monogram")
    m.addGenConstrIndicator(use_y3, True, y3 >= 40, name="Activate_Monogram")
    m.addGenConstrIndicator(use_y3, False, y3 == 0, name="Deactivate_Monogram")

    # Solve the model
    m.optimize()

    # Store costs for each scenario, weighted by scenario probability
    if m.status == GRB.OPTIMAL:
        perfect_foresight_costs.append(m.objVal * s['p'])

# Calculate WS (Wait-and-See)
WS = sum(perfect_foresight_costs)

# SP is the expected cost using the original stochastic solution
SP = 11343.5  # from your original calculation

# Calculate EVPI
EVPI = SP - WS

print("SP (Stochastic Programming Solution):", SP)
print("WS (Wait-and-See Solution):", WS)
print("EVPI (Expected Value of Perfect Information):", EVPI)

TypeError: 'float' object is not subscriptable

In [53]:
import gurobipy as gp
from gurobipy import GRB

# Given data setup
probabilities = [0.09, 0.12, 0.10, 0.05, 0.16, 0.14, 0.03, 0.08, 0.05, 0.05, 0.04, 0.03, 0.02, 0.01, 0.02, 0.01]
demands = [90, 95, 100, 105, 110, 115, 120, 125, 130, 135, 140, 145, 150, 155, 160, 165]
supplier_costs = [120, 105, 110]  # Costs for Phil & Sebastian, Rosso, and Monogram respectively
advance_order_cost = 95

# Calculate mean demand
mean_demand = sum(p * d for p, d in zip(probabilities, demands))

# Model setup for the deterministic mean value problem
m_mean = gp.Model("Deterministic_Mean_Value_Problem")

# Decision variables
x_mean = m_mean.addVar(name="x_mean", vtype=GRB.CONTINUOUS)  # Base order

# Objective function to minimize cost
m_mean.setObjective(advance_order_cost * x_mean, GRB.MINIMIZE)

# Constraint to ensure demand is met using average demand
m_mean.addConstr(x_mean >= mean_demand, "AverageDemand")

# Solve the model
m_mean.optimize()

print("Objective for the Average:", m_mean.objVal)

# Step 2: stochastic solution holding first-stage variables fixed
# Create a new optimization model
m_stochastic = gp.Model("Stochastic_Model")

# Use first-stage decisions from the mean value problem
fs_x = x_mean.X

# Scenario-specific decision variables
y = m_stochastic.addVars(3, 16, vtype=GRB.CONTINUOUS, name="y")  # Emergency orders

# Objective function to minimize total expected cost
objective = gp.quicksum(probabilities[n] * (advance_order_cost * fs_x + sum(supplier_costs[i] * y[i, n] for i in range(3))) for n in range(16))
m_stochastic.setObjective(objective, GRB.MINIMIZE)

# Constraints for each scenario
for n in range(16):
    m_stochastic.addConstr(fs_x + gp.quicksum(y[i, n] for i in range(3)) >= demands[n], name=f"Demand_{n}")

# Solve our model
m_stochastic.optimize()

print("Objective for the Stochastic Solution:", m_stochastic.objVal)

# EEV from the mean value problem
EEV = m_mean.objVal

# SP from the stochastic model
SP = m_stochastic.objVal

# Calculate VSS
VSS = abs(EEV - SP)

print("EEV (Expected Ex-Post Value):", EEV)
print("SP (Expected Value of Stochastic Solution):", SP)
print("VSS (Value of Stochastic Solution):", VSS)


Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.0.0 23A344)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1 rows, 1 columns and 1 nonzeros
Model fingerprint: 0xea7caddd
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+02, 1e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+02, 1e+02]
Presolve removed 1 rows and 1 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.0891750e+04   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds (0.00 work units)
Optimal objective  1.089175000e+04
Objective for the Average: 10891.75
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.0.0 23A344)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 16 rows, 48